In [ ]:
!pip uninstall torch-xla

In [ ]:
import pandas as pd
import os
from transformers import DistilBertTokenizerFast
from transformers import Trainer, TrainingArguments
import json
from pathlib import Path

# Загрузка и тест модели

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
model.device

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

device(type='cpu')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
'''


prompt = """ЗАДАЧА
Напиши программу, которая выводит "Hello, World!"
НЕПРАВИЛЬНОЕ РЕШЕНИЕ
pront('Hello, World!')
ПРАВИЛЬНОЕ РЕШЕНИЕ
print('Hello, World!)
В ответе укажи исправления, которые необходимо внести.
Будь сух, уверен и немногословен, старайся уместить ответ в одно-два предложения, чем короче, тем лучше. Не используй слово "ученик"
НИ ЗА ЧТО НЕ ВЫДАВАЙ ПРАВИЛЬНЫЙ ВАРИАНТ ОТВЕТА, ТОЛЬКО НАМЕКАЙ.
"""
messages = [
    {"role": "system", "content":
     """Ты - учитель по программированию на PYTHON. Твоя цель - анализировать неправильные решения задач, и указывать исправления, которые необходимо внести. Задачи требуют решения на python, однако ученики могут по ошибке отправить решение на другом языке программирования.
     В каждом запросе будет приведено УСЛОВИЕ ЗАДАЧИ, НЕПРАВИЛЬНОЕ РЕШЕНИЕ и образец ПРАВИЛЬНОГО РЕШЕНИЯ.
     НИ ЗА ЧТО НЕ ВЫДАВАЙ ПРАВИЛЬНЫЙ ВАРИАНТ ОТВЕТА, ТОЛЬКО НАМЕКАЙ."""
     },
      {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response'''

'\n\n\nprompt = """ЗАДАЧА\nНапиши программу, которая выводит "Hello, World!"\nНЕПРАВИЛЬНОЕ РЕШЕНИЕ\npront(\'Hello, World!\')\nПРАВИЛЬНОЕ РЕШЕНИЕ\nprint(\'Hello, World!)\nВ ответе укажи исправления, которые необходимо внести.\nБудь сух, уверен и немногословен, старайся уместить ответ в одно-два предложения, чем короче, тем лучше. Не используй слово "ученик"\nНИ ЗА ЧТО НЕ ВЫДАВАЙ ПРАВИЛЬНЫЙ ВАРИАНТ ОТВЕТА, ТОЛЬКО НАМЕКАЙ.\n"""\nmessages = [\n    {"role": "system", "content":\n     """Ты - учитель по программированию на PYTHON. Твоя цель - анализировать неправильные решения задач, и указывать исправления, которые необходимо внести. Задачи требуют решения на python, однако ученики могут по ошибке отправить решение на другом языке программирования.\n     В каждом запросе будет приведено УСЛОВИЕ ЗАДАЧИ, НЕПРАВИЛЬНОЕ РЕШЕНИЕ и образец ПРАВИЛЬНОГО РЕШЕНИЯ.\n     НИ ЗА ЧТО НЕ ВЫДАВАЙ ПРАВИЛЬНЫЙ ВАРИАНТ ОТВЕТА, ТОЛЬКО НАМЕКАЙ."""\n     },\n      {"role": "user", "content": prompt}\n]\ntext = tok

# Подготовка датасета

In [ ]:
HM_answers = json.load(open('sortme-data-Llama-405B-answers.json', encoding='utf-8'))
sort_me_data = json.load(open('sort_me_round_solutions_new.json', encoding='utf-8'))

In [ ]:
template = '''Ты должен выступить в роли учителя: проанализировать и прокомментировать ошибки в решении простой алгоритмической задачи.
В ответе укажи исправления, которые необходимо внести.
Будь уверен, старайся уместить ответ в одно-два предложения, чем короче, тем лучше.

УСЛОВИЕ ЗАДАЧИ:
{task}

ОБРАЗЕЦ правильного решения:
{correct_example}

НЕПРАВИЛЬНОЕ РЕШЕНИЕ:
{student_solution}

Вердикт тест-системы:
{verdict}

НИ ЗА ЧТО НЕ ВЫДАВАЙ ПРАВИЛЬНЫЙ ВАРИАНТ ОТВЕТА, ТОЛЬКО НАМЕКАЙ.
ТВОЙ ОТВЕТ:
{answer}
'''

In [ ]:
data = []
answers = []
for task in sort_me_data:
    task_id = str(task['task_id'])
    if task_id  in HM_answers:
        for i, solution in enumerate(task['solutions']):
            if i < len(HM_answers[task_id]):
                prompt = template.format(
                    task=task['description'],
                    correct_example=task['right_solution'],
                    student_solution=solution['code'],
                    verdict=solution['verdict'],
                    answer=HM_answers[task_id][i]['author_comment']
                )
                data.append(prompt)

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer,mlm=False)

In [ ]:
from sklearn.model_selection import train_test_split
train_, test_ = train_test_split(data,test_size=0.2)

In [ ]:
train = [string for string in train_]
test = [string for string in test_]

In [ ]:
train = tokenizer(train, padding=True, return_tensors='pt')
test = tokenizer(test, padding=True, return_tensors='pt')

In [ ]:
!pip install datasets

In [ ]:
import datasets
'''def test_gen():
  for i in range(len(test['input_ids'])):
    yield {'input_ids':test['input_ids'][i], 'attention_mask': test['attention_mask']}
def train_gen():
  for i in range(len(train['input_ids'])):
    yield {'input_ids':train['input_ids'][i], 'attention_mask': train['attention_mask']}'''
test = datasets.Dataset.from_dict(test)
train = datasets.Dataset.from_dict(train)

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    'output',
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8, disable_tqdm=False
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=train,
    eval_dataset=test
)

In [ ]:
trainer.train()

KeyboardInterrupt: 

In [ ]:
trainer.save_model()